## Importing The Required Modules And Functions

In [1]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

## Download The Source File From The Cloud

In [2]:
# !wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

## Extract The Zip Files

In [3]:
import zipfile
with zipfile.ZipFile('D:\PycharmProjects\GettingStartedWithSQL\source.zip', 'r') as zip_ref:
    zip_ref.extractall('D:\PycharmProjects\GettingStartedWithSQL\source_data')

## Set The Path For The Target Files

In [4]:
tmpfile    = "source_temp.tmp"               # store all extracted data

logfile    = "source_logfile.txt"            # all event logs will be stored

targetfile = "source_transformed_data.csv"   # transformed data is stored

## CSV Extract Function

In [5]:
def extract_from_csv(file_to_process): 
    dataframe = pd.read_csv(file_to_process) 
    return dataframe

## JSON Extract Function

In [6]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

## XML Extract Function

In [7]:
def extract_from_xml(file_to_process):

    dataframe = pd.DataFrame(columns=['name','height','weight'])

    tree = ET.parse(file_to_process) 

    root = tree.getroot() 

    for person in root: 

        name = person.find("name").text 

        height = float(person.find("height").text)

        weight = float(person.find("weight").text) 

        dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index=True) 

        return dataframe

## Extract Function

In [8]:
def extract():
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("source_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("source_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("source_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data

## Transform

In [9]:
def transform(data):
        #Convert height which is in inches to millimeter
        #Convert the datatype of the column into float
        #data.height = data.height.astype(float)
        #Convert inches to meters and round off to two decimals(one inch is 0.0254 meters)
        data['height'] = round(data.height * 0.0254,2)
        
        #Convert weight which is in pounds to kilograms
        #Convert the datatype of the column into float
        #data.weight = data.weight.astype(float)
        #Convert pounds to kilograms and round off to two decimals(one pound is 0.45359237 kilograms)
        data['weight'] = round(data.weight * 0.45359237,2)
        return data

## Loading

In [10]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile) 

## Logging

In [11]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running The ETL Process

In [12]:
log("ETL Job Started")

In [13]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30
5,alex,65.78,112.99
6,ajay,71.52,136.49
7,alice,69.40,153.03
8,ravi,68.22,142.34
9,joe,67.79,144.30


In [14]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data 

,name,height,weight
0,alex,1.67,51.25
1,ajay,1.82,61.91
2,alice,1.76,69.41
3,ravi,1.73,64.56
4,joe,1.72,65.45
5,alex,1.67,51.25
6,ajay,1.82,61.91
7,alice,1.76,69.41
8,ravi,1.73,64.56
9,joe,1.72,65.45


In [15]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [16]:
log("ETL Job Ended")